In [ ]:
!pip install snscrape

In [55]:
import pandas as pd
import numpy as np
import snscrape.modules.twitter as sntwitter
import datetime

In [56]:
def fetcher(comp, start, end):
  query = f'${comp} until:{end} since:{start}'
  lis = []
  limit = 50
  for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(lis) == limit:
      break
    else:
      try:
        if tweet.lang == 'en' and tweet.cashtags == [comp] and tweet.likeCount >= 1:
          lis.append([start,tweet.rawContent,tweet.viewCount,tweet.replyCount,tweet.retweetCount,tweet.likeCount])
      except:
        pass
  data = pd.DataFrame(lis,columns= ['date','content','viwes','replycount','retweetCount','likeCount'])
  return data

In [52]:
data = fetcher('AMZN','2018-01-02','2018-01-03 ')

In [ ]:
stock = pd.read_csv('/content/drive/MyDrive/MC data/stock_data.csv',index_col = 0)
stock

In [67]:
temp = list(stock.iloc[:,0])
temp1 = []
temp2 = []
arr = []
for i in range(len(temp)):
  x = temp[i][:10]
  arr.append(int(x[:4]))
  arr.append(int(x[5:7]))
  arr.append(int(x[8:10]))
  x = datetime.date(arr[0],arr[1],arr[2])
  temp1.append(x)
  x = x+datetime.timedelta(days=1)
  temp2.append(x)
stock['Next_Day'] = temp2
stock.iloc[:,0] = temp1

In [ ]:
stock

In [69]:
stock['Score'] = 1
temp = []
names = stock.iloc[:,7].unique()
for i in names:
  df = stock[stock.iloc[:,7]==i]
  a = (df.iloc[:,4] - df.iloc[:,1])/df.iloc[:,1]
  b = min(a)
  if b<0:
    a+= -b
  a /= max(a)
  temp += list(a)
stock['Score'] = temp

stock.head()

,0,1,2,3,4,5,6,7,Next_Day,Score
0,2018-01-02,112.040001,116.110001,112.040001,114.480003,114.480003,187400.0,ICLR,2018-01-03,0.552252
1,2018-01-02,114.489998,114.940002,113.660004,114.019997,114.019997,379800.0,ICLR,2018-01-03,0.425530
2,2018-01-02,114.339996,114.739998,112.650002,113.910004,113.910004,422700.0,ICLR,2018-01-03,0.427217
3,2018-01-02,114.220001,117.980003,113.629997,117.620003,117.620003,299000.0,ICLR,2018-01-03,0.591367
4,2018-01-02,117.669998,117.989998,115.680000,117.370003,117.370003,277000.0,ICLR,2018-01-03,0.433147


In [ ]:
stock['Sentiment'] = "Neutral"
scores = temp.copy()
temp = []

for i in range(len(stock)):
  if scores[i]>0.6:
    temp.append('Positive')

  elif scores[i]<0.4:
    temp.append('Negative')
  
  else:
    temp.append('Neutral')
  
  print(i)
stock['Sentiment'] = temp

In [71]:
stock = stock.drop(stock.columns[1:7],axis = 1)

In [74]:
stock

,Start_Day,Stock,Next_Day,Score,Sentiment
0,2018-01-02,ICLR,2018-01-03,0.552252,Neutral
1,2018-01-02,ICLR,2018-01-03,0.425530,Neutral
2,2018-01-02,ICLR,2018-01-03,0.427217,Neutral
3,2018-01-02,ICLR,2018-01-03,0.591367,Neutral
4,2018-01-02,ICLR,2018-01-03,0.433147,Neutral
...,...,...,...,...,...
219061,2018-01-02,JBHT,2018-01-03,0.510253,Neutral
219062,2018-01-02,JBHT,2018-01-03,0.499409,Neutral
219063,2018-01-02,JBHT,2018-01-03,0.378713,Negative
219064,2018-01-02,JBHT,2018-01-03,0.536161,Neutral


In [73]:
stock.columns = ['Start_Day','Stock','Next_Day','Score','Sentiment']

In [17]:
for i in stock['Stock'].unique():
  main_data = pd.DataFrame([],columns= ['date','content','viwes','replycount','retweetCount','likeCount'])
  df = stock[stock['Stock'] == i]
  temp = []
  for j in range(len(df)):
    data = fetcher(df.iloc[j,1],df.iloc[j,0],df.iloc[j,2])
    main_data = pd.concat([main_data, data], ignore_index=True)
  else:
    main_data.to_csv(f'/content/drive/MyDrive/MC data/{i}_data.csv')

array(['ICLR', 'LI', 'AVGO', 'LRCX', 'UAL', 'EXPD', 'ADP', 'INCY', 'ENPH',
       'ZS', 'TCOM', 'GRMN', 'VTRS', 'TSCO', 'SGEN', 'TTD', 'UTHR',
       'EBAY', 'AZPN', 'MAR', 'BIIB', 'ARGX', 'NDAQ', 'GILD', 'FITB',
       'ASML', 'DXCM', 'NBIX', 'VRSK', 'PFG', 'KLAC', 'FTNT', 'LNT',
       'SBAC', 'TTWO', 'NVDA', 'MRVL', 'SIVB', 'FISV', 'TMUS', 'ILMN',
       'TECH', 'CSGP', 'TEAM', 'NTAP', 'EA', 'MCHP', 'ORLY', 'WMG',
       'NVCR', 'MDLZ', 'TRMB', 'CTAS', 'LKQ', 'MU', 'FDS', 'GOOG', 'ROST',
       'EXPE', 'HOLX', 'ADSK', 'CPRT', 'BMRN', 'EXC', 'REGN', 'KHC',
       'ALNY', 'ANSS', 'CSX', 'NFLX', 'MELI', 'AMGN', 'IBKR', 'TROW',
       'PDD', 'XEL', 'WBA', 'POOL', 'INTU', 'CSCO', 'SAN', 'VRTX', 'FSLR',
       'AXON', 'PCAR', 'NXPI', 'ULTA', 'ERIE', 'PYPL', 'AMAT', 'CHTR',
       'ISRG', 'CDNS', 'NDSN', 'NTRS', 'PODD', 'FOXA', 'AAPL', 'SNPS',
       'FWONK', 'AZN', 'FAST', 'MSFT', 'DDOG', 'PAYX', 'HZNP', 'LPLA',
       'GMAB', 'ABNB', 'TER', 'INTC', 'CDW', 'AMZN', 'FANG', 'IDXX',
       '

In [18]:
df = stock[stock['Stock'] == 'ICLR']
type(df)

pandas.core.frame.DataFrame

In [ ]:
stock.to_csv('stock_data.csv',index = False)